In [64]:
import ee
import geemap
import geopandas as gpd
import numpy as np

In [2]:
# ee.Authenticate()
ee.Initialize()

In [6]:
# Open source table as a dataframe
camps_gdf = gpd.read_file(r"C:/Users/jedid/Documents/GIS/agricultural_camps_centroids.gpkg", layer="agricultural_camps_centroids")

# Preview the data
camps_gdf.head()

,province,district,block,camp,area_sqkm,area_ha,other_ fea,region,rainfall,growth_per,x,y,geometry
0,Copperbelt,Mufulira,Murundu,Mokambo,59.0,5870.0,None,III,Above 1000mm,120-150 days,28.328695,-12.455062,POINT (28.32869 -12.45506)
1,Copperbelt,Kalulushi,Chati,Chati Central,209.0,20862.0,None,III,Above 1000mm,120-150 days,27.802077,-12.774708,POINT (27.80208 -12.77471)
2,Copperbelt,Kalulushi,Chati,Kafubu Depot,244.0,24414.0,None,III,Above 1000mm,120-150 days,27.879169,-12.832888,POINT (27.87917 -12.83289)
3,Copperbelt,Lufwanyama,Kafubu,Lumanto,98.0,9795.0,None,III,Above 1000mm,120-150 days,27.748971,-12.956329,POINT (27.74897 -12.95633)
4,Copperbelt,Lufwanyama,Kafubu,Chikabuke,84.0,8350.0,None,III,Above 1000mm,120-150 days,27.740611,-13.041749,POINT (27.74061 -13.04175)


In [42]:
selected_camps_gdf = camps_gdf[camps_gdf['camp'] == 'Kafubu Depot'].copy()
selected_camps_gdf

,province,district,block,camp,area_sqkm,area_ha,other_ fea,region,rainfall,growth_per,x,y,geometry
2,Copperbelt,Kalulushi,Chati,Kafubu Depot,244.0,24414.0,None,III,Above 1000mm,120-150 days,27.879169,-12.832888,POINT (27.87917 -12.83289)


In [20]:
# Convert GDF to list of EE Features
features = []
for _, row in selected_camps_gdf.iterrows():
    # geom = ee.Geometry.Polygon(row['geometry'].exterior.coords)  # for polygons
    geom = ee.Geometry.Point(row['geometry'].x, row['geometry'].y)
    feature = ee.Feature(geom)
    features.append(feature)

In [80]:
# Convert to EE FeatureCollection
fc = ee.FeatureCollection(features)

# Load raster image
image = ee.Image("ISDASOIL/Africa/v1/sand_content") \
    .clip(fc) \
    .select('mean_0_20') \
    # .rename('Carbon Organic')

# sampled = image.divide(100)

# Sample the raster using the FeatureCollection
# You can also use `.reduceRegions()` if you want zonal stats
sampled = image.sampleRegions(
    collection=fc,
    scale=30,
    geometries=True
)

convert = np.exp(sampled.getInfo()['features'][0]['properties']['mean_0_20'] / 10) - 1

print(convert)
# print(sampled.getInfo()['features'][0]['properties'])

491.7490410932563


In [81]:
# Print some results
print(sampled.getInfo()['features'][0]['properties'])

# Export to Google Drive
# task = ee.batch.Export.table.toDrive(
#     collection=sampled,
#     description='sampled_values',
#     fileFormat='CSV'
# )
# task.start()

{'mean_0_20': 62}
